### Importing Pandas and Numpy

In [ ]:
import pandas as pd
import numpy as np

### Reading File that has region data and title id

In [ ]:
tsv_file = "title.akas.tsv"

with open(tsv_file, "r") as f:
    output = f.read()
df = pd.read_table(tsv_file, sep="\t")   
df.head(2)    
f.close()

In [ ]:
#Drop null values in region column

df1 = df.dropna(axis=0,subset=['region'])

In [ ]:
df1.dropna(axis=0,inplace=True)

In [ ]:
len(df1['region'].unique())

### Importing IMDB csv file

In [ ]:
imdb = pd.read_csv('imdb2.csv')

### Importing tmdb csv file

In [ ]:
tmdb = pd.read_csv('tmdb2.csv')

In [ ]:
#Rename column to making merging dataframes easier

imdb = imdb.rename(columns = {'tconst':'titleId'})

### Merging tsv dataframe with imdb dataframe

In [ ]:
new_df2 = pd.merge(imdb,df1, on="titleId",how="left")

In [ ]:
#Only keep rows where region is equal to "US"
new_df3 = new_df2[new_df2['region'] =="US"]

In [ ]:
#name change
imdb2 = new_df3

In [ ]:
#Cleaning up by dropping columns
imdb3 = imdb2.drop(["types","attributes","isOriginalTitle"],axis=1)

In [ ]:
#Checking to see if both title columns contain the same info... they don't
imdb3['primary_title'].equals(imdb3['title'])

In [ ]:
#Creating new column to display where two title columns don't match
imdb3['movie_title'] = np.where(imdb3['primary_title'] == imdb3['title'], imdb3['title'],"different")

In [ ]:
#rows where "primary_title" and "title" do not match
imdb3.loc[imdb3['movie_title']=="different"]

In [ ]:
#dropping new column and "primary_title column" so that one title column remains
imdb3.drop(['primary_title','movie_title'], axis=1, inplace=True)

### Cleaning up tmdb file to prepare for merging

In [ ]:
tmdb.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
#creating new column to see if two title columns match... they do not
tmdb['movie_title'] = np.where(tmdb['original_title'] == tmdb['title'], tmdb['title'],"different")

In [ ]:
#dropping two title columns so that one remains
tmdb.drop(['original_title','movie_title'], axis=1, inplace=True)

### Right merge on title column

In [ ]:
merge_test = pd.merge(imdb3,tmdb, on="title",how="right")

In [ ]:
#removing language columns - not needed
merge_test.drop(['original_language','language'], axis=1, inplace=True)

In [ ]:
#filling null "genre" values to see if I can pull in the genres based on "genre_id" column
merge_test["genres"].fillna("no genre", inplace = True) 

### Importing movie budget csv

In [ ]:
money = pd.read_csv("tn.movie_budgets.csv")

In [ ]:
#renaming column to make merging simpler
money.rename(columns = {'movie': 'title'}, inplace = True)

In [ ]:
money.head()

### Left merge on title

In [ ]:
all_dfs = pd.merge(merge_test, money, on="title",how="left")

In [ ]:
#still trying to figure out genre column

no_genre = all_dfs[(all_dfs["genres"]=="no genre")]
len(no_genre)

In [ ]:
all_dfs.head()

In [ ]:
all_dfs.to_csv("combined_movie_data.csv")